In [1]:
# Standard library imports
import os
import random
import pickle

# Data analysis packages
import numpy as np
import pandas as pd
from scipy.signal import correlate, correlation_lags
from scipy.ndimage import gaussian_filter1d

# Visualization packages
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyArrow, Patch, Circle
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D
from matplotlib import gridspec

# Image processing packages
import imageio
from PIL import Image, ImageSequence

# Braingeneers packages for analysis
import braingeneers
import braingeneers.data.datasets_electrophysiology as ephys
from braingeneers.analysis.analysis import SpikeData, read_phy_files, load_spike_data, burst_detection, randomize_raster

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import gaussian_filter1d
from human_hip  import spike_data

In [2]:
hc_328 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated_s1.zip')
hc_52 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-05-10-e-hc52_18790_unperturbed/derived/kilosort2/hc5.2_chip18790_baseline_rec5.10.23_curated_s1.zip')
hc_111 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2022-11-02-e-Hc11.1-chip16753/derived/kilosort2/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_curated.zip')
hc_112823 = spike_data.read_autocurated_data('/workspaces/human_hippocampus/data/ephys/2023-12-03-e-Hc112823_avv9hckcr1/derived/kilosort2/Hc112723_hckcr1_21841_120323_base_acqm.zip')
hc_112823_2 = spike_data.read_autocurated_data('/workspaces/human_hippocampus/data/ephys/2023-12-03-e-Hc112823_avv9hckcr1/derived/kilosort2/Hc112723_hckcr1_21841_120323_base2_acqm.zip')
hc_120523_chip21769 = read_phy_files("/workspaces/human_hippocampus/data/ephys/2023-12-11-e-Hc120523_aav9hckcr1_chip21769/elliott/elliott_chip21769_base.zip")

/tmp/ipykernel_62554/35721603.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  hc_328 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated_s1.zip')
/tmp/ipykernel_62554/35721603.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  hc_52 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-05-10-e-hc52_18790_unperturbed/derived/kilosort2/hc5.2_chip18790_baseline_rec5.10.23_curated_s1.zip')
/tmp/ipykernel_62554/35721603.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  hc_111 = read_phy_files('/workspaces/human_hippocampus/data/ephys/2022-11-02-e-Hc11.1-chip16753/derived/k

In [21]:
def count_total_events(spikedata):
    """
    Counts the total number of spike events in a spikedata object.
    
    Parameters:
    - spikedata: A spikedata object with an attribute 'train', which is a list of spike trains.
    
    Returns:
    - n_events: The total number of spike events across all spike trains in the spikedata object.
    """
    n_events = 0
    for train in spikedata.train:
        n_events += len(train)
    
    return n_events

def get_summary(dataset):
    return {
        "Number of Spikes": count_total_events(dataset),
        "Length (s)": dataset.length / 1000,
        "Number of Neurons": dataset.N
    }

# Your dataset paths
datasets = {
    "hc_328": '/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated_s1.zip',
    "hc_52": '/workspaces/human_hippocampus/data/ephys/2023-05-10-e-hc52_18790_unperturbed/derived/kilosort2/hc5.2_chip18790_baseline_rec5.10.23_curated_s1.zip',
    "hc_111": '/workspaces/human_hippocampus/data/ephys/2022-11-02-e-Hc11.1-chip16753/derived/kilosort2/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_curated.zip',
    "hc_112823": '/workspaces/human_hippocampus/data/ephys/2023-12-03-e-Hc112823_avv9hckcr1/derived/kilosort2/Hc112723_hckcr1_21841_120323_base_acqm.zip',
    "hc_112823_2": '/workspaces/human_hippocampus/data/ephys/2023-12-03-e-Hc112823_avv9hckcr1/derived/kilosort2/Hc112723_hckcr1_21841_120323_base2_acqm.zip',
    "hc_120523_chip21769": "/workspaces/human_hippocampus/data/ephys/2023-12-11-e-Hc120523_aav9hckcr1_chip21769/elliott/elliott_chip21769_base.zip"
}

# A dictionary to hold all summaries
summaries = {}

# Go through each dataset, load it, and get its summary
for label, path in datasets.items():
    if 'acqm' in path:
        dataset = spike_data.read_autocurated_data(path)
    else:
        dataset = read_phy_files(path)  # Assuming this function exists and loads your data
    summary = spike_data.text_summary(dataset)
    summaries[label] = summary

# Convert the summaries dictionary to a DataFrame
df_summaries = pd.DataFrame.from_dict(summaries, orient='index')


/tmp/ipykernel_62554/2685933559.py:42: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  dataset = read_phy_files(path)  # Assuming this function exists and loads your data


Number of spikes:  103920
Length:  398 seconds
Number of Neurons:  136
Average Firing Rate:  1.92
Coefficient of Variation:  0.872


/tmp/ipykernel_62554/2685933559.py:42: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  dataset = read_phy_files(path)  # Assuming this function exists and loads your data


Number of spikes:  21747
Length:  107 seconds
Number of Neurons:  100
Average Firing Rate:  2.02
Coefficient of Variation:  0.595


/tmp/ipykernel_62554/2685933559.py:42: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  dataset = read_phy_files(path)  # Assuming this function exists and loads your data


Number of spikes:  49667
Length:  418 seconds
Number of Neurons:  87
Average Firing Rate:  1.36
Coefficient of Variation:  0.546
Number of spikes:  338136
Length:  313 seconds
Number of Neurons:  249
Average Firing Rate:  4.34
Coefficient of Variation:  0.585
Number of spikes:  153743
Length:  126 seconds
Number of Neurons:  351
Average Firing Rate:  3.46
Coefficient of Variation:  0.61


/tmp/ipykernel_62554/2685933559.py:42: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  dataset = read_phy_files(path)  # Assuming this function exists and loads your data


Number of spikes:  36481
Length:  300 seconds
Number of Neurons:  51
Average Firing Rate:  2.38
Coefficient of Variation:  0.69


In [ ]:
#Complete the 'findMedian' function below.
#
# The function is expected to return an INTEGER.
# The function accepts INTEGER_ARRAY arr as parameter.

def findMedian(arr):
    # Write your code here
    arr.sort()
    n = len(arr)
    if n % 2 == 0:
        return (arr[n // 2] + arr[n // 2 - 1]) // 2
    return arr[n // 2]

In [23]:
df_summaries

,0
hc_328,None
hc_52,None
hc_111,None
hc_112823,None
hc_112823_2,None
hc_120523_chip21769,None


In [ ]:
import pandas as pd

# Function to summarize a dataset
def get_summary(spike_data, dataset_name):
    summary = spike_data.text_summary(dataset_name)
    # Assuming `text_summary()` returns a dictionary, if not you'll need to parse the string
    summary['Dataset'] = dataset_name  # Add the dataset name to the summary
    return summary

# Initialize a list to hold your summary dictionaries
summaries = []

# List of datasets and their paths
datasets = {
    "hc_328": '/path/to/hc_328_data',
    "hc_52": '/path/to/hc_52_data',
    # Add all other datasets here
}

# Loop through the datasets, read and summarize each one
for dataset_name, path in datasets.items():
    data = read_phy_files(path)  # Or the appropriate function for reading your data
    summary = get_summary(data, dataset_name)
    summaries.append(summary)

# Create a DataFrame from the summaries
summary_df = pd.DataFrame(summaries)

# View the DataFrame
print(summary_df)


In [7]:
spike_data.text_summary(hc_328)

Number of spikes:  103920
Length:  398 seconds
Number of Neurons:  136
Average Firing Rate:  1.92
Coefficient of Variation:  0.872


In [9]:
spike_data.text_summary(hc_52)

Number of spikes:  21747
Length:  107 seconds
Number of Neurons:  100
Average Firing Rate:  2.02
Coefficient of Variation:  0.595


In [15]:
spike_data.text_summary(hc_111)

Number of spikes:  49667
Length:  418 seconds
Number of Neurons:  87
Average Firing Rate:  1.36
Coefficient of Variation:  0.546


In [18]:
spike_data.text_summary(hc_112823)

Number of spikes:  338136
Length:  313 seconds
Number of Neurons:  249
Average Firing Rate:  4.34
Coefficient of Variation:  0.585


In [20]:
spike_data.text_summary(hc_112823_2)

Number of spikes:  153743
Length:  126 seconds
Number of Neurons:  351
Average Firing Rate:  3.46
Coefficient of Variation:  0.61


In [22]:
spike_data.text_summary(hc_120523_chip21769)

Number of spikes:  36481
Length:  300 seconds
Number of Neurons:  51
Average Firing Rate:  2.38
Coefficient of Variation:  0.69


Number of spikes:  103920
Length:  398 seconds
Number of Neurons:  136
Average Firing Rate:  1.92
Coefficient of Variation:  0.872


AttributeError: 'NoneType' object has no attribute 'append'